In [46]:
from __future__ import print_function
import json
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def load_data(file):
    """Загрузка входных данных из файла"""
    with open(file, 'r') as f:
        input_data = json.load(f)
    couriers = {}
    orders = {}
    points = {}
    for depotData in input_data['depots']:
        points[depotData['point_id']] = {
            'location': [depotData['location_x'], depotData['location_y']],
            'timewindow': [0, 1439],
        }
    for courierData in input_data['couriers']:
        couriers[courierData['courier_id']] = {
            'location': [courierData['location_x'], courierData['location_y']],
            'time': 360,
        }
    for orderData in input_data['orders']:
        points[orderData['pickup_point_id']] = {
            'location': [orderData['pickup_location_x'], orderData['pickup_location_y']],
            'timewindow': [orderData['pickup_from'], orderData['pickup_to']],
            'order_time': {orderData['order_id']: orderData['pickup_from']}
        }
        points[orderData['dropoff_point_id']] = {
            'location': [orderData['dropoff_location_x'], orderData['dropoff_location_y']],
            'timewindow': [orderData['dropoff_from'], orderData['dropoff_to']],
        }
        orders[orderData['order_id']] = orderData
    return couriers, orders, points

def get_travel_duration_minutes(location1, location2):
    """Время перемещения курьера от точки location1 до точки location2 вминутах"""
    distance = abs(location1[0] - location2[0]) + abs(location1[1] - location2[1])
    return 10 + distance

def get_current_courier_reward(time, payment):
    work_payment = time * 2
    profit = orders_payment - work_payment
    return profit

def delete_element(array, idx):
    result = []
    for i in range(len(array)):
        if not(i == idx):
            result.append(array[i])
    return result

In [47]:
tmp = []
tmp.append(((data[1][10001]['pickup_from'], data[1][10001]['pickup_to']), 
          (data[1][10002]['pickup_from'], data[1][10002]['pickup_to'])))

In [48]:
tmp

[((360, 1380), (360, 1380))]

In [93]:
load_data("data/hard_input.json")[0]

{1: {'location': [20, 20], 'time': 360},
 2: {'location': [50, 50], 'time': 360},
 3: {'location': [150, 20], 'time': 360},
 4: {'location': [25, 150], 'time': 360},
 5: {'location': [100, 300], 'time': 360}}

In [77]:
couriers_start_position = [data[0][i+1]['location'] for i in range(len(data[0]))]
points_pickup = np.array([[data[1][i+10001]['pickup_location_x'],
                           data[1][i+10001]['pickup_location_y']] for i in range(len(data[1]))])
time_pickup = np.array([[data[1][10001+i]['pickup_from'],
                         data[1][10001+i]['pickup_to']] for i in range(len(data[1]))])
time_dropoff = np.array([[data[1][10001+i]['dropoff_from'],
                          data[1][10001+i]['dropoff_to']] for i in range(len(data[1]))])    
points_dropoff = np.array([[data[1][i+10001]['dropoff_location_x'],
                            data[1][i+10001]['dropoff_location_y']] for i in range(len(data[1]))])
payments = np.array([data[1][10001+i]['payment'] for i in range(len(data[1]))])
pickup_id = np.array([data[1][10001+i]['pickup_point_id'] for i in range(len(data[1]))])
dropoff_id = np.array([data[1][10001+i]['dropoff_point_id'] for i in range(len(data[1]))])

In [78]:
len(data[1])

5

In [79]:
from tqdm import tqdm_notebook

def get_by_id(point_id):
    if point_id < 10000:
        return couriers_start_position[point_id]
    if point_id < 60000:
        return points_pickup[point_id-40001]
    else:
        return points_dropoff[point_id-60001]

def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
    return vector 


def get_directions_table(pickup_ids):
    directions_table = []
    for pickup_id in pickup_ids:
        n = pickup_id - 40001
        n_pickup_in_table = len(data[0]) + n
        n_dropoff_in_table = len(data[0]) + len(data[1]) + n 
        directions_table.append([n_pickup_in_table, n_dropoff_in_table])
    return directions_table

def get_time_windows(pickup_id, dropoff_id, n_couriers):
    tmp_string = np.concatenate((np.array(range(n_couriers)), 
                                 pickup_id, 
                                 dropoff_id))
    tmp_couriers = [(360,1440) for i in range(n_couriers)]
    tmp_pickup = []
    tmp_dropoff = []
    for i in tqdm_notebook(range(len(pickup_id))):
        tmp_pickup.append((data[1][pickup_id[i] - 30000]['pickup_from'], 
                           data[1][pickup_id[i]-30000]['pickup_to']))
        tmp_dropoff.append((data[1][dropoff_id[i] - 50000]['dropoff_from'], 
                           data[1][dropoff_id[i]-50000]['dropoff_to']))
    result = []
    for i in tqdm_notebook(range(len(tmp_couriers))):
        result.append(tmp_couriers[i])
    for i in range(len(tmp_pickup)):
        result.append(tmp_pickup[i])
    for i in range(len(tmp_dropoff)):
        result.append(tmp_dropoff[i])
    return result
    
def get_time_windows(pickup_id, dropoff_id, n_couriers):
    tmp_string = np.concatenate((np.array(range(n_couriers)), 
                                 pickup_id, 
                                 dropoff_id))
    tmp_couriers = [(360,1440) for i in range(n_couriers)]
    tmp_pickup = []
    tmp_dropoff = []
    for i in tqdm_notebook(range(len(pickup_id))):
        tmp_pickup.append((data[1][pickup_id[i] - 30000]['pickup_from'], 
                           data[1][pickup_id[i]-30000]['pickup_to']))
        tmp_dropoff.append((data[1][dropoff_id[i] - 50000]['dropoff_from'], 
                           data[1][dropoff_id[i]-50000]['dropoff_to']))
    result = []
    for i in tqdm_notebook(range(len(tmp_couriers))):
        result.append(tmp_couriers[i])
    for i in range(len(tmp_pickup)):
        result.append(tmp_pickup[i])
    for i in range(len(tmp_dropoff)):
        result.append(tmp_dropoff[i])
    return result


def create_distance_matrix(pickup_id, dropoff_id, n_couriers):
    tmp_string = np.concatenate((np.array(range(n_couriers)), 
                                 pickup_id, 
                                 dropoff_id))
    n_pickup = len(data[1])
    n_dropoff = len(data[1])
    matrix = []
    for i in tqdm_notebook(range(len(tmp_string))):
        tmp = []
        for j in range(len(tmp_string)):
            tmp.append(get_travel_duration_minutes(get_by_id(tmp_string[j]), 
                                                   get_by_id(tmp_string[i])))
            
            
        matrix.append(tmp)
    return np.pad(matrix, 1, pad_with)[:-1,:-1]

In [80]:
time_windows = get_time_windows(pickup_id, dropoff_id, n_couriers=len(data[0]))

In [81]:
dist=get_directions_table(pickup_id)

In [82]:
len(data[1])

5

In [83]:
data[1]

{10001: {'order_id': 10001,
  'pickup_point_id': 40001,
  'pickup_location_x': 20,
  'pickup_location_y': 100,
  'pickup_from': 360,
  'pickup_to': 1380,
  'dropoff_point_id': 60001,
  'dropoff_location_x': 60,
  'dropoff_location_y': 110,
  'dropoff_from': 360,
  'dropoff_to': 1180,
  'payment': 900},
 10002: {'order_id': 10002,
  'pickup_point_id': 40002,
  'pickup_location_x': 55,
  'pickup_location_y': 100,
  'pickup_from': 360,
  'pickup_to': 1380,
  'dropoff_point_id': 60002,
  'dropoff_location_x': 63,
  'dropoff_location_y': 130,
  'dropoff_from': 560,
  'dropoff_to': 1380,
  'payment': 900},
 10003: {'order_id': 10003,
  'pickup_point_id': 40003,
  'pickup_location_x': 16,
  'pickup_location_y': 104,
  'pickup_from': 420,
  'pickup_to': 720,
  'dropoff_point_id': 60003,
  'dropoff_location_x': 67,
  'dropoff_location_y': 150,
  'dropoff_from': 360,
  'dropoff_to': 720,
  'payment': 900},
 10004: {'order_id': 10004,
  'pickup_point_id': 40004,
  'pickup_location_x': 66,
  'pick

In [84]:
matrix = create_distance_matrix(pickup_id, dropoff_id, len(data[0]))

In [85]:
[(360,1440) for i in range(3)]

[(360, 1440), (360, 1440), (360, 1440)]

In [86]:
n_couriers = len(data[0])

In [87]:
def create_data_model(matrix, dist, n_couriers, time_windows):
    d = {'distance_matrix': matrix, 
         'pickups_deliveries':dist,
        'num_vehicles':n_couriers,
        'depot':0,
         'time_windows':time_windows}
    return d
    # [END data_model]


In [88]:
create_data_model(matrix, dist, n_couriers, time_windows)

{'distance_matrix': array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0],
        [  0,  10,  70, 140, 145, 370,  90, 125,  98, 128, 156, 140, 163,
         187, 201, 225],
        [  0,  70,  10, 140, 135, 310,  90,  65,  98,  68, 136,  80, 103,
         127, 141, 165],
        [  0, 140, 140,  10, 265, 340, 220, 185, 228, 166,  26, 190, 207,
         223,  77,  95],
        [  0, 145, 135, 265,  10, 235,  65,  90,  65, 109, 261,  85,  68,
          52, 198, 208],
        [  0, 370, 310, 340, 235,  10, 290, 255, 290, 252, 336, 240, 217,
         193, 273, 283],
        [  0,  90,  90, 220,  65, 290,  10,  45,  18,  64, 216,  60,  83,
         107, 153, 163],
        [  0, 125,  65, 185,  90, 255,  45,  10,  53,  29, 181,  25,  48,
          72, 118, 128],
        [  0,  98,  98, 228,  65, 290,  18,  53,  10,  72, 224,  60,  83,
         107, 161, 171],
        [  0, 128,  68, 166, 109, 252,  64,  29,  72,  10, 162,  34,  51,
          69,  9

In [90]:
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(matrix, dist, n_couriers, time_windows)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # Define cost of each arc.
    # [START arc_cost]
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Add Distance constraint.
    # [START distance_constraint]
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        1440,  # no slack
        500,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    # [END distance_constraint]
    
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        300,  # allow waiting time
        3000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        print(time_window[0], time_window[1])
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))
        
    # Define Transportation Requests.
    # [START pickup_delivery_constraint]
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))
    # [END pickup_delivery_constraint]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    assignment = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if assignment:
        print_solution(data, manager, routing, assignment)
    # [END print_solution]


if __name__ == '__main__':
    main()
# [END program]

360 1440
360 1440
360 1440
360 1440
360 1380
360 1380
420 720
600 1200
480 1380
360 1180
560 1380
360 720
800 1400
960 1380
Route for vehicle 0:
 0 ->  6 ->  8 ->  7 ->  11 ->  12 ->  13 ->  4 -> 0
Distance of the route: 215m

Route for vehicle 1:
 0 ->  5 ->  10 -> 0
Distance of the route: 336m

Route for vehicle 2:
 0 ->  2 ->  9 ->  14 ->  15 -> 0
Distance of the route: 201m

Route for vehicle 3:
 0 ->  1 -> 0
Distance of the route: 0m

Route for vehicle 4:
 0 ->  3 -> 0
Distance of the route: 0m

Total Distance of all routes: 752m
